In [32]:
import torch
from Net import Net
from Data import Data
from Train import train_free_energy_two_inputs
import numpy as np
from scipy import integrate
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
torch.manual_seed(3)
%matplotlib qt

#### Make some data

In [10]:
points = 499
lower = 0.00001
upper = 2*np.pi + lower
data_set = Data(points, lower, upper)
x, y = data_set.get()
matrix = data_set.matrix()
simpson_matrix = data_set.simpson_matrix()
input_var = data_set.input_variables()
output_dict = {}

In [11]:
Layers = [2, 16, 16, 16, 1]

In [12]:
learning_rate = 0.02
rho_list = np.linspace(3.25, 6, 12, endpoint=True)
for rho in rho_list:
    net = Net(Layers)
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.9)
    lambd = 200
    net = train_free_energy_two_inputs(net, x, input_var, rho, lambd, scheduler,
                                   simpson_matrix ,optimizer, matrix, epochs = 20000,
                                   pre_training=True, diagram=False)
    output = net(input_var)
    factor = integrate.simps(output.data.numpy().T, x.data.numpy().T)
    output = output.data.numpy() * (1/factor)
    output_dict["rho" + str(rho)] = output

In [13]:
rho = 5.5
net = Net(Layers)
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.9)
lambd = 200
net = train_free_energy_two_inputs(net, x, input_var, rho, lambd, scheduler,
                               simpson_matrix ,optimizer, matrix, epochs = 20000,
                               pre_training=True, diagram=False)
output = net(input_var)
factor = integrate.simps(output.data.numpy().T, x.data.numpy().T)
output = output.data.numpy() * (1/factor)
output_dict["rho" + str(rho)] = output

In [14]:
for rho in rho_list:
    plt.plot(x.data.numpy(), output_dict["rho" + str(rho)], label= "$\\rho_{0} = $" + str(rho))
    plt.xlabel("$\\theta$")
    plt.ylabel("$f(\\theta)$")
    plt.ylim([0, 0.6])
    plt.title("distribution function for rodlike molecule")
    plt.legend(loc= 'upper right')
    plt.show()
    plt.savefig('figure/' + 'rho=' + str(rho) + '.pdf')

plot 3D diagram

In [15]:
print (output_dict.keys())

dict_keys(['rho3.25', 'rho3.5', 'rho3.75', 'rho4.0', 'rho4.25', 'rho4.5', 'rho4.75', 'rho5.0', 'rho5.25', 'rho5.5', 'rho5.75', 'rho6.0'])


In [103]:
plt.plot(x.data.numpy(), output_dict["rho" + str(4.25)])

In [30]:
s = 5.5
output_dict["rho" + str(s)] = np.flipud(output_dict["rho" + str(s)])
for i in range(len(output_dict["rho" + str(s)])):
    if output_dict["rho" + str(s)][i] < output_dict["rho" + str(s)][i+1]:
        first = output_dict["rho" + str(s)][0: i]
        second = output_dict["rho" + str(s)][i:]
        final = np.append(second, first)
        output_dict["rho" + str(s)] =
        break


In [44]:
s = 6.0
output_dict["rho" + str(s)] = output_dict["rho" + str(s)].reshape(-1, 1)

In [58]:
height = np.concatenate([output_dict["rho" + str(3.25)],
                        output_dict["rho" + str(3.5)],
                        output_dict["rho" + str(3.75)],
                        output_dict["rho" + str(4.0)],
                        output_dict["rho" + str(4.25)],
                        output_dict["rho" + str(4.5)],
                        output_dict["rho" + str(4.75)],
                        output_dict["rho" + str(5.0)],
                        output_dict["rho" + str(5.25)],
                        output_dict["rho" + str(5.5)],
                        output_dict["rho" + str(5.75)],
                        output_dict["rho" + str(6.0)]],
                        axis = 1)
fig = plt.figure()
ax = Axes3D(fig)
X, Y = np.meshgrid(rho_list, x.data.numpy())
ax.plot_surface(X, Y, height, rstride=1, cstride=1, cmap=plt.get_cmap('rainbow'))
ax.set_xlabel('$\\rho_{0}$')
ax.set_ylabel('$\\theta$')
ax.set_zlabel('$f(\\theta)$')
plt.show()

ValueError: shape mismatch: objects cannot be broadcast to a single shape